# 第2部 - パーティのためのMNISTを用いたWML Fedareted Learning

### 学習目標

パート2 - WML Federated Learning with MNIST for Partyを修了すると，以下のことができるようになります．

- Federated Learning実験で使用するデータをロードする。
- IBM Federated Learning ライブラリをインストールする。
- データハンドラを定義する。データハンドラの詳細については、<a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">データハンドラのカスタマイズ</a>を参照してください。
- アグリゲータでデータを訓練するパーティを設定します。



<div class="alert-block alert-info">このノートは、Federated Learning実験の管理者または接続者が実行することを想定しています。
</div>

## 目次

- [1. データを読み込む](#load)
    - [1.1 MNIST 手書き数字データセットのダウンロード](#1.1)
- [2. Fedarated Learingライブラリのインストール](#install)
    - [2.1 FL付きIBM WML SDK をインストール](#2.1)
    - [2.2 ライブラリのインストール](#2.2)
    - [2.3 フレームワークのインストール](#2.3)
    - [2.4 パーティのインポート](#2.4)
- [3. データハンドラの定義](#data-handler)
- [4. パーティの設定](#config)
- [5. Fedarated Learningで学習する](#train)
- [6. まとめ](#summary)

<div class="alert-block alert-warning">このノートブックを実行する前に、すでにPart 1 - WML Federated Learning with MNIST for Adminを実行している必要があります。実行していない場合は、ノートブックを開いて、まずそのノートブックを実行してください。
</div> 

<a id = "load"></a>
## 1. データを読み込む

### Part 1 Notebookからの変数の貼り付け
Part 1のNotebookの最後に取得したID認証情報を貼り付けます。Part 1を実行していない場合は、まずNotebookを開いて実行してください。

In [ ]:
WML_SERVICES_HOST = 'us-south.ml.cloud.ibm.com'
IAM_APIKEY = 'xxx'
RTS_ID = 'xxx'
TRAINING_ID = 'xxx'

<a id = "1.1"></a>
### 1.1 MNIST 手書き数字データセットのダウンロード

Partyとして、統合学習モデルの学習に使用するデータセットを提供する必要があります。  
このチュートリアルでは、デフォルトでMNISTの手書き数字データセットが提供されます。

In [ ]:
import requests

dataset_resp = requests.get("https://api.dataplatform.cloud.ibm.com/v2/gallery-assets/entries/903188bb984a30f38bb889102a1baae5/data",
                            allow_redirects=True)

f = open('MNIST-pkl.zip', 'wb')
f.write(dataset_resp.content)
f.close()

In [ ]:
import zipfile
import os

with zipfile.ZipFile("MNIST-pkl.zip","r") as file:
    file.extractall()
    
!ls -lh

<a id = "install"></a> </a>
## 2. Federated Learning ライブラリのインストール
ここでは、PythonクライアントでFederated Learningを呼び出すために必要なライブラリなどのパッケージをインストールしていきます。

<a id = "2.1"></a> </a>
### 2.1 FL付きIBM WML SDK をインストール
IBM Watson Machine Learning CLI を、Federated Learningを含んだソフトウェア開発パッケージ全体と一緒にインストールします。

In [ ]:
!pip install --upgrade ibm-watson-machine-learning | tail -n 1

<a id = "2.2"></a> 
### 2.2 ライブラリのインストール

In [ ]:
!pip install environs parse websockets jsonpickle pandas pytest pyYAML requests pathlib2 psutil setproctitle tabulate lz4 opencv-python gym ray==0.8.0 cloudpickle==1.3.0 image | tail -n 1

<a id = "2.3"></a>
### 2.3 フレームワークのインストール

In [ ]:
!pip install tensorflow==2.1.0 scikit-learn==0.23.1 keras==2.2.4 numpy==1.17.4 scipy==1.4.1 | tail -n 1

<a id = "2.4"></a>
### 2.4 パーティのインポート
以下のコードは、パーティ用のパッケージをインポートし、それがロードされることを確認します。

In [ ]:
import ibmfl.party_env_validator
from ibmfl.party.party import Party

<a id = "data-handler"></a>
## 3. データハンドラの定義
パーティはデータセットが互換性のあるフォーマットで一貫性のあるものであることを保証するためにデータハンドラを実行する必要があります。このチュートリアルでは、MNISTデータセットのデータハンドラの例を示します。

データハンドラの詳細については、<a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-cus-dh.html?audience=wdp&context=cpdaas" target="_blank" rel="noopener no referrer">データハンドラのカスタマイズ</a>を参照してください。

このデータハンドラは、このノートブックのローカル作業ディレクトリに書き込まれます。

In [ ]:
%%writefile mnist_keras_data_handler.py
from keras.preprocessing.image import ImageDataGenerator
import logging

import numpy as np
from keras.utils import np_utils

from ibmfl.data.data_handler import DataHandler
from ibmfl.util.datasets import load_mnist

logger = logging.getLogger(__name__)



class MnistTFDataHandler(DataHandler):
    """
       Data handler for MNIST dataset.
       """

    def __init__(self, data_config=None, channels_first=False):
        super().__init__()
        self.file_name = None
        if data_config is not None:
            if 'train_file' in data_config:
                self.train_file_name = data_config['train_file']
            if 'test_file' in data_config:
                self.test_file_name = data_config['test_file']

    def get_data(self, nb_points=500):
        """
        Gets pre-process mnist training and testing data. Because this method
        is for testing it takes as input the number of datapoints, nb_points,
        to be included in the training and testing set.

        :param: nb_points: Number of data points to be included in each set
        :type nb_points: `int`
        :return: training data
        :rtype: `tuple`
        """
        if self.file_name is None:
            (x_train, y_train), (x_test, y_test) = load_mnist()
            # Reduce datapoints to make test faster
            x_train = x_train[:nb_points]
            y_train = y_train[:nb_points]
            x_test = x_test[:nb_points]
            y_test = y_test[:nb_points]
        else:
            try:
                logger.info(
                    'Loaded training data from ' + str(self.file_name))
                data_train = np.load(self.file_name)
                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_train, y_train)= pickle.load(f)

                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_test, y_test)= pickle.load(f)
                
            except Exception:
                raise IOError('Unable to load training data from path '
                              'provided in config file: ' +
                              self.file_name)

        # Add a channels dimension
        import tensorflow as tf
        x_train = x_train[..., tf.newaxis]
        x_test = x_test[..., tf.newaxis]

        print('x_train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        print(x_test.shape[0], 'test samples')

        return (x_train, y_train), (x_test, y_test)

### データハンドラが存在することを確認する

In [ ]:
!ls -lh

<a id = "config"></a>
## 4. パーティーを構成する

各パーティは、アグリゲータに呼び出すために、パーティ設定ファイルを実行する必要があります。以下にパーティ構成の例を示します。

このノートブックの前のセクションでトレーニングID、RTS ID、データハンドラを既に定義しており、ローカルトレーニングとプロトコルハンドラは全てSDKで定義されているので、データセットファイルの情報は`["data"]["info"]`の下で定義するだけでよいでしょう。

このチュートリアルでは、前のセクションで紹介した examplar MNIST データセットを読み込んでいるので、データパスはすでに定義されています。

In [ ]:
from pathlib import Path
working_dir = !pwd
pwd = working_dir[0]

party_config = {
  "aggregator": {
    "ip": WML_SERVICES_HOST + "/ml/v4/trainings/" + TRAINING_ID
  },
  "connection": {
    "info": {
      "id": RTS_ID,
    }
  },
  "data": {
    "info": {
      "train_file": "/mnist-keras-train.pkl",
      "test_file": "/mnist-keras-test.pkl"
    },
    "name": "MnistTFDataHandler",
    "path": pwd + "/mnist_keras_data_handler.py"
  },
  "local_training": {
    "name": "LocalTrainingHandler",
    "path": "ibmfl.party.training.local_training_handler"
  },
  "protocol_handler": {
    "name": "PartyProtocolHandler",
    "path": "ibmfl.party.party_protocol_handler"
  }
}

In [ ]:
print(party_config)

<a id = "train"></a>
## 5. フェデレーテッド・ラーニングに接続してトレーニングする

ここでようやくアグリゲータに接続してトレーニングを始めることができます。

#### クラウド認証トークンの取得

In [ ]:
from ibm_watson_machine_learning import APIClient


wml_credentials = {
    "apikey": IAM_APIKEY,
    "url": "https://" + WML_SERVICES_HOST
}

wml_client = APIClient(wml_credentials)
IAMTOKEN = "Bearer " + wml_client.wml_token
print(IAMTOKEN)

### 5.1 アグリゲータへの接続を確立する

In [ ]:
p = Party( config_dict = party_config, token = IAMTOKEN )

``Received Heartbeat from Aggregator``というメッセージが表示されたら、パーティーの開始準備が整いました。

### 5.2 トレーニングの開始

In [ ]:
p.start()

<a id = "summary"></a> 
## まとめ

おめでとう！あなたは以下のことを学びました。

1. Fedareted Learningの実験を開始する
2. テンプレートモデルを読み込む
3. RTSを作成し、実験ジョブを起動する
4. 学習用のデータセットをロードする
5. データハンドラの定義
6. パーティーを設定する
7. アグリゲータに接続する
8. Federated Learningモデルをトレーニングする

### もっと詳しく知りたい方はこちら

- Federated Learningの設定、用語、UIからのFederated Learningの実行についての詳細は、<a href = "https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp" target="_blank" rel="noopener no referrer">Federated Learning documentation</a> for Cloudを参照してください。
- Kerasのモデルテンプレートの詳細については、<a href = "https://www.tensorflow.org/tutorials/quickstart/advanced" target="_blank" rel="noopener no referrer">こちら</a>を参照してください。
